<a href="https://colab.research.google.com/github/lizzifer991/Big-Data_Challenge/blob/main/Level-1/Lvl1_Digital_EBooks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Import Dependencies
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()


Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 256 kB in 3s (97.5 kB/s)
Reading 

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-09-13 06:20:21--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.2’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.18MB/s    in 0.2s    

2022-09-13 06:20:21 (4.18 MB/s) - ‘postgresql-42.2.9.jar.2’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Books").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Ebook_Purchase_v1_01.tsv.gz"
spark.sparkContext.addFile(url)

In [5]:
ebook_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Digital_Ebook_Purchase_v1_01.tsv.gz"), sep="\t", header=True)

#Show DataFrame
ebook_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   33605939| RGYFDX8QXKEIR|B007KO2MLO|     328837464|           Big Maria|Digital_Ebook_Pur...|          4|            0|          0|   N|                N|              Quirky|Elmore Leonard me...| 2013-09-09|
|         US|   34058393|R13CBGTMNV9R8Z|B005FLODDE|     764276359|The Woman Who Was...|Digital_Ebook_Pur

In [6]:
#Count Records
ebook_df.count()

5101693

In [7]:
#DropNA's
ebook_df = ebook_df.dropna()

In [8]:
#DropDuplicates
ebook_df = ebook_df.dropDuplicates()

In [9]:
#Count Records
ebook_df.count()

5101476

In [10]:
#Create a Review_df
Review_df= ebook_df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
#DropDuplicates
Review_df= Review_df.dropDuplicates()
#DropNA's
Review_df= Review_df.dropna()
#Show Review_df
Review_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R10167B0GOQ4E5|   44487646|B008ED5EAG|     868031612| 2012-09-07|
|R101KPSASAHPBU|   37456071|B009I4IQVE|     472909892| 2013-03-07|
|R101Q8QQRNE7GQ|   53096441|B003UV8XHS|     687621661| 2012-11-13|
|R101ZD7HQ1JAD2|   11298774|B005EH750C|     420831159| 2012-01-13|
|R1023OHYBOELYT|   44877462|B007WU2S5Q|     186652133| 2013-07-20|
|R102BCG1R2S46T|   44352456|B00BD9HOY2|     637989477| 2013-03-03|
|R102DPLUNIVSU5|   34856820|B0049H8WS2|     553702651| 2013-03-20|
|R102SAQ1UMJSC0|   10186325|B00AP4WRDY|     842201773| 2012-12-30|
|R102SBKTZO39D9|   11410950|B007EUYVPE|     585356695| 2013-08-21|
|R102VFO6V4MT3W|   51665978|B007A2RMQQ|     942933542| 2013-01-13|
|R10339RCIMXQQC|   16540449|B0098QU6K6|     319368183| 2012-09-24|
|R103IWNMEXLY57|   40492897|B009QE04HU|     139399120| 2012-11

In [11]:
Review_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- review_date: string (nullable = true)



In [12]:
#Create a Product_df
Product_df= ebook_df.select(["product_id","product_title"])
#DropDuplicates
Product_df= Product_df.dropDuplicates()
#DropNA's
Product_df= Product_df.dropna()
#Show Review_df
Product_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00A5MRG6O|          Dad Is Fat|
|B004GUSG4M|Before I Go To Sl...|
|B002WLCKG2|Trial by Fire: A ...|
|B00F1KWDZO|    Elvis Sunny Bear|
|B004GB1FIK|Cassidy Jones and...|
|B00ALPRZXE|Please Don't Tick...|
|B0064J2B9Y|How To Get Your E...|
|B00EA9SELE|Stitch (Slasherto...|
|B00CL4HZIS|Binary Cycle: Dis...|
|B00DND202W|He Loves Lucy (Th...|
|B007FEF9C8|Death of a Hero: ...|
|B00F0ZZVZY|Basic Ballet Move...|
|B0076BT83Q|CARDIO SUCKS: The...|
|B001MSMULG|The Kill Artist (...|
|B004I8VGYO|Becoming Odyssa: ...|
|B0013TTJSG|Wild Card: An Eli...|
|B00ETR7S8M|Divine Interventi...|
|B00CMB9OV6|Profile of Evil (...|
|B00B4HAI2I|With the Lightnin...|
|B00CYB8CD0|Time to converse ...|
+----------+--------------------+
only showing top 20 rows



In [13]:
Product_df.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



In [14]:
#Create a Customers_df
Customers_df= ebook_df.groupby("customer_id")\
              .agg({"customer_id": "count"})\
              .withColumnRenamed("count(customer_id)", "customer_count")
#DropDuplicates
Customers_df= Customers_df.dropDuplicates()
#DropNA's
Customers_df= Customers_df.dropna()
#Show Review_df
Customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   18961431|             3|
|   47259344|            15|
|   37819413|             4|
|   12713078|            10|
|   53062366|             1|
|   16196522|            58|
|    6694836|             1|
|   50878423|            17|
|   52079566|             1|
|   46513898|             3|
|   17495907|             3|
|   15809627|             1|
|   46813332|             1|
|   52100766|             2|
|   13347298|             9|
|   24730566|             1|
|   12064992|            22|
|   28427418|            93|
|   44187661|            27|
|   17636075|            14|
+-----------+--------------+
only showing top 20 rows



In [15]:
Customers_df.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- customer_count: long (nullable = false)



In [16]:
#Count Review Records
Review_df.count()

5101476

In [17]:
#Count Product Records
Product_df.count()

649977

In [18]:
#Count Customer Records
Customers_df.count()

2062116

In [19]:
# Create vine_table
vine_table_df = ebook_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R100EWUFY7SJUQ|          5|            1|          1|   N|
|R100L9BE6NM6SL|          3|            0|          0|   N|
|R100P4FGV3WQZ2|          5|            0|          0|   N|
|R1018ZQC1CVCN5|          5|            3|          4|   N|
|R101KZBKN7XJFL|          4|            3|          4|   N|
|R101O0UJK4501C|          5|            0|          0|   N|
|R101PP4OFK6EEI|          5|           25|         25|   N|
|R101R6K1VEQ1K1|          5|            5|          5|   N|
|R101T352MIPBZM|          5|            1|          1|   N|
|R101VSTX15JFTS|          5|            0|          0|   N|
|R101XSBXOHOUZT|          5|            0|          0|   N|
|R1026UQ8DPSCAM|          5|            4|          4|   N|
|R102ADSQC6Z7HE|          5|            0|          0|   N|
|R102IBTE8OWLRV|          5|            

In [20]:
#Configure for RDS
mode = "append"
jdbc_url="jdbc:postgresql://mypostgresdb.cvwouuigjbgo.us-east-2.rds.amazonaws.com:5432/amazon_reviews_us_Digital_Ebook_Purchase_v1_01.tsv.gz"
config = {"user":"root", 
          "password": "Cucumber#88", 
          "driver":"org.postgresql.Driver"}

In [21]:
#Write Review_df to RDS
Review_df.write.jdbc(url=jdbc_url, table="review_id_table", mode=mode, properties=config)

Py4JJavaError: ignored

In [ ]:
#Write Product_df to RDS
Product_df.write.jdbc(url=jdbc_url, table="products", mode=mode, properties=config)

In [ ]:
#Write Customer_df to RDS
Customers_df.write.jdbc(url=jdbc_url, table="customers", mode=mode, properties=config)

In [ ]:
#Write Vine_table_df to RDS
vine_table_df.write.jdbc(url=jdbc_url, table="vine_table", mode=mode, properties=config)